In [2]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
import os.path as osp
from calibrate_table import calibrate_table_by_aruco
from table_markers import get_table_markers_coords_in_table_frame
from utils import show, get_image_from_camera
from params import box_size, table_aruco_size, K, D, top_camera_id, front_camera_id

In [ ]:
# visualize table frame

read_image_from_file = True
if read_image_from_file:
    use_top_image = True
    if use_top_image:
        view = "top"
        image_file = "images/calibrate_table/aruco/top/0000.jpg"
    else:
        view = "front"
        image_file = "images/calibrate_table/aruco/front/0000.jpg"
    image = cv2.imread(image_file)
else:
    view = "top"
    image = get_image_from_camera(top_camera_id)

table_frame, _ = calibrate_table_by_aruco(image, view, K, D, table_aruco_size)
if table_frame is None:
    print("Could not detect table")
else:
    camera2table = table_frame.origin2plane()
    rvec, _ = cv2.Rodrigues(camera2table[0:3, 0:3])
    tvec = camera2table[0:3, 3]
    draw = image.copy()
    cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.1)
    show(draw)

In [ ]:
# get table aruco corners in table frame

read_images_from_files = True
if read_images_from_files:
    view = "top"
    folder = "images/calibrate_table/aruco/top"
    image_files = list(map(lambda f: osp.join(folder, f), sorted(os.listdir(folder))))
    images = [cv2.imread(image_file) for image_file in image_files]
else:
    view = "top"
    n = 5
    images = [get_image_from_camera(top_camera_id) for i in range(n)]

corners_3d_in_table_all = list()
for image in images:
    table_frame, corners_3d_in_camera = \
        calibrate_table_by_aruco(image, view, K, D, table_aruco_size)
    corners_3d_in_camera = corners_3d_in_camera[:, 0, 0, :]
    # corners_3d_in_camera.shape = (n, 3)
    corners_3d_in_table = table_frame.to_plane(corners_3d_in_camera)
    corners_3d_in_table_all.append(corners_3d_in_table)

corners_3d_in_table_all = np.array(corners_3d_in_table_all)
corners_3d_in_table_mean = corners_3d_in_table_all.mean(axis=0)

print(corners_3d_in_table_mean.shape)
print(corners_3d_in_table_mean)

In [4]:
np.save('data/aruco_corners_in_table.npy', corners_3d_in_table_mean)

In [ ]:
# get table markers coords in table frame

read_image_from_file = True
if read_image_from_file:
    view = "top"
    image_file = "images/calibrate_table/markers/0000.jpg"
    image = cv2.imread(image_file)
else:
    view = "top"
    image = get_image_from_camera(top_camera_id)

table_frame, _ = calibrate_table_by_aruco(image, view, K, D, table_aruco_size)
target_table_markers = \
    get_table_markers_coords_in_table_frame(image, view, K, D, table_frame, box_size / 2)

print(target_table_markers)

In [ ]:
np.save('data/target_table_markers.npy', target_table_markers)